In [1]:
# install the folium library
!pip install folium
# install the geopy library
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# import libraries
import time
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
import folium

In [3]:
# list of cities, first will be used as reference city
cities = ['caldas da rainha', 'viseu', 'coimbra', 'leiria', 'tomar', 'santarém','aveiro','soure','odemira', 'lagoa', 'faro']

geolocator = Nominatim(user_agent='ipt_big_data_rui_shoaib') # create a Nominatim geolocator specifying a user agent
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1) # pass it through a rate limiter to not exceed limits imposed by Nominatim (1 request/s)

cities_df = pd.DataFrame(columns=['city', 'latitude', 'longitude']) # create a dataframe to hold the data

# iterate through each city
for city in cities:
    raw = geocode(city) # geocode with a delay form the ratelimiter
    if raw:
        cities_df = pd.concat([cities_df, pd.DataFrame([{
            'city': city,
            'latitude': raw.latitude,
            'longitude': raw.longitude,
        }])], ignore_index=True) # append to dataframe
    else:
        print(f'Error: failed to geocode {city}')

cities_df # display dataframe

,city,latitude,longitude
0,caldas da rainha,39.407212,-9.134600
1,viseu,40.657471,-7.913866
2,coimbra,40.211193,-8.429463
3,leiria,39.743790,-8.807112
4,tomar,39.603660,-8.415049
5,santarém,39.236364,-8.686708
6,aveiro,40.640496,-8.653784
7,soure,40.059503,-8.626390
8,odemira,37.597600,-8.642200
9,lagoa,37.132581,-8.455051


In [28]:
ref_city = cities_df.iloc[0] # first city as ref

for i, row in cities_df.iterrows():
    ref_coor = (ref_city['latitude'], ref_city['longitude']) # ref city coordinates
    cur_coor = (row['latitude'], row['longitude']) # current city coordinates
    dist = distance.distance(ref_coor, cur_coor).km # calculates distance between ref city & current city in km
    cities_df.at[i, 'ref_city'] = ref_city['city'] # it's a good idea to keep track of ref city individually by adding a col, in case we use multiple ref cities in the future
    cities_df.at[i, 'dist_to_ref_km'] = dist # add a col with the distance

ref_city = cities_df.iloc[0] # update ref city

cities_df.to_json('cities_df.json') # export to json

cities_df # display dataframe

,city,latitude,longitude,ref_city,dist_to_ref_km
0,caldas da rainha,39.407212,-9.134600,caldas da rainha,0.000000
1,viseu,40.657471,-7.913866,caldas da rainha,173.570354
2,coimbra,40.211193,-8.429463,caldas da rainha,107.769685
3,leiria,39.743790,-8.807112,caldas da rainha,46.778038
4,tomar,39.603660,-8.415049,caldas da rainha,65.616910
5,santarém,39.236364,-8.686708,caldas da rainha,43.029108
6,aveiro,40.640496,-8.653784,caldas da rainha,142.955977
7,soure,40.059503,-8.626390,caldas da rainha,84.517103
8,odemira,37.597600,-8.642200,caldas da rainha,205.417436
9,lagoa,37.132581,-8.455051,caldas da rainha,259.393558


In [31]:
map = folium.Map(location=[39.3, -8], zoom_start=7) # create a map object

def getMarkerPopup(row, reference=False):
    return """
<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
</head>
<body>

<h4>"""+row['city'].title()+"""</h4>
""" + ("<h5>(Reference City)</h5>" if reference else "")+ """

<table>
  <tr>
    <th>Latitude</th>
    <th>Longitude</th>
    <th>Reference City</th>
    <th>Distance to Reference City</th>
  </tr>
  <tr>
    <td>""" + str(row['latitude']) + """</td>
    <td>""" + str(row['longitude']) + """</td>
    <td>""" + row['ref_city'].title() + """</td>
    <td>{:.2f}""".format(row['dist_to_ref_km']) + """ KM</td>
  </tr>
</table>

</body>
</html>
    """

folium.Marker(
    location=(ref_city['latitude'], ref_city['longitude']),
    popup=getMarkerPopup(ref_city, True),
    icon=folium.Icon(color='green')
).add_to(map) # add reference city marker and make it stand out

# iterate over the dataframe, [1:] skips first row bcz it's the ref city
for i, row in cities_df[1:].iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=getMarkerPopup(row)).add_to(map) # add a marker for each city to the map

map # display the map

In [17]:
openweathermap_api_key = '2c7a57835bdb1afab7ea8699e8b319d4' # openweathermap api keys take ~2 hours to be activated, until then a 401 will be received

# run every hour indefintely until interrupted forcefully
# while True:
#     for i, row in cities_df.iterrows():
#         # https://openweathermap.org/current
#         res = requests.get(f'https://api.openweathermap.org/data/2.5/weather?lat={row["latitude"]}&lon={row["longitude"]}&appid={openweathermap_api_key}')
#         print(res)
#     time.sleep(60)

<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>


KeyboardInterrupt: 